The traffic accident data is the key to this analysis. To make sure it's current, instead of downloading a static .csv file it, along with several other datasets from the City of Nashville's OpenData portal, will be periodically downloaded and updated via APIs.

To avoid too many pull requests, all of the data will be pulled here and the relevant information will be exported for use in a separate notebook for EDA.

** This notebook will be revisited and edited as data needs may change for this analysis.

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
# Needed to pip install sodapy first
from sodapy import Socrata
from geopy.geocoders import Nominatim

**Data Source:** https://data.nashville.gov/Police/Traffic-Accidents/6v6w-hpcw/about_data

In [2]:
client = Socrata("data.nashville.gov", None)

In [3]:
crashes_export = client.get("6v6w-hpcw", limit=200000)
crashes_raw = pd.DataFrame.from_records(crashes_export)
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,reporting_officer,collision_type,collision_type_description,weather,...,rpa,precinct,lat,long,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,property_damage
0,20240148168,2024-03-02T18:20:00.000,2,0,0,True,807344,11,Front to Rear,21,...,9531,HERMIT,36.1889,-86.6283,"{'type': 'Point', 'coordinates': [-86.6283, 36...",1,2,7,44,NaN
1,20240148129,2024-03-02T18:46:00.000,2,1,0,False,320820,11,Front to Rear,21,...,8651,MIDTOW,35.9961,-86.6888,"{'type': 'Point', 'coordinates': [-86.6888, 35...",1,2,29,37,NaN
2,20240148065,2024-03-02T18:05:00.000,2,3,0,False,4000938,4,ANGLE,21,...,9209,HERMIT,36.2037,-86.6956,"{'type': 'Point', 'coordinates': [-86.6956, 36...",1,1,8,38,NaN
3,20240148045,2024-03-02T17:55:00.000,2,0,0,False,4007311,11,Front to Rear,21,...,2133,MADISO,36.2694,-86.7564,"{'type': 'Point', 'coordinates': [-86.7564, 36...",1,2,16,16,NaN
4,20240148007,2024-03-02T00:00:00.000,2,0,0,False,4010860,4,ANGLE,22,...,9105,HERMIT,36.1517,-86.677,"{'type': 'Point', 'coordinates': [-86.677, 36....",1,1,8,38,NaN


In [4]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180885 entries, 0 to 180884
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   accident_number              180885 non-null  object
 1   date_and_time                180885 non-null  object
 2   number_of_motor_vehicles     180884 non-null  object
 3   number_of_injuries           180885 non-null  object
 4   number_of_fatalities         180885 non-null  object
 5   hit_and_run                  180871 non-null  object
 6   reporting_officer            180874 non-null  object
 7   collision_type               180875 non-null  object
 8   collision_type_description   180875 non-null  object
 9   weather                      174824 non-null  object
 10  weather_description          174824 non-null  object
 11  illuaccidemination           180603 non-null  object
 12  illumination_description     180603 non-null  object
 13  harmfulcodes  

Now that the DataFrame has been created, some columns can be removed as they're not relevant to this analysis.

In [5]:
crashes_raw = crashes_raw.drop(crashes_raw.columns[[7, 9, 11, 13, 23, 24, 25, 26, 27]], axis=1)

In [6]:
crashes_raw['date'] = pd.to_datetime(crashes_raw['date_and_time']).dt.date
crashes_raw = crashes_raw.astype({'number_of_motor_vehicles': 'float',
                    'number_of_injuries': 'float',
                    'number_of_fatalities': 'float',
                    'hit_and_run': 'bool',
                    'property_damage': 'bool'})

In [7]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180885 entries, 0 to 180884
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   accident_number             180885 non-null  object 
 1   date_and_time               180885 non-null  object 
 2   number_of_motor_vehicles    180884 non-null  float64
 3   number_of_injuries          180885 non-null  float64
 4   number_of_fatalities        180885 non-null  float64
 5   hit_and_run                 180885 non-null  bool   
 6   reporting_officer           180874 non-null  object 
 7   collision_type_description  180875 non-null  object 
 8   weather_description         174824 non-null  object 
 9   illumination_description    180603 non-null  object 
 10  harmfuldescriptions         178804 non-null  object 
 11  street_address              180879 non-null  object 
 12  city                        180885 non-null  object 
 13  state         

For some initial cleaning, all text fields should be converted to the same case, in this case UPPER

In [8]:
crashes_raw['collision_type_description'] = crashes_raw['collision_type_description'].astype(str).str.upper()
crashes_raw['illumination_description'] = crashes_raw['illumination_description'].astype(str).str.upper()
crashes_raw['harmfuldescriptions'] = crashes_raw['harmfuldescriptions'].astype(str).str.upper()
crashes_raw['street_address'] = crashes_raw['street_address'].astype(str).str.upper()
crashes_raw['city'] = crashes_raw['city'].astype(str).str.upper()
crashes_raw['state'] = crashes_raw['state'].astype(str).str.upper()
crashes_raw['precinct'] = crashes_raw['precinct'].astype(str).str.upper()
crashes_raw['weather_description'] = crashes_raw['weather_description'].astype(str).str.upper()
crashes_raw['property_damage'] = crashes_raw['property_damage'].astype(str).str.upper()

In [9]:
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,reporting_officer,collision_type_description,weather_description,illumination_description,...,street_address,city,state,zip,rpa,precinct,lat,long,property_damage,date
0,20240148168,2024-03-02T18:20:00.000,2.0,0.0,0.0,True,807344,FRONT TO REAR,CLEAR,DARK - LIGHTED,...,LEBANON PKE & HICKORY HILL LN,HERMITAGE,TN,37076,9531,HERMIT,36.1889,-86.6283,TRUE,2024-03-02
1,20240148129,2024-03-02T18:46:00.000,2.0,1.0,0.0,False,320820,FRONT TO REAR,CLEAR,DARK - LIGHTED,...,US HWY 31 ALT & PETTUS RD,NASHVILLE,TN,37211,8651,MIDTOW,35.9961,-86.6888,TRUE,2024-03-02
2,20240148065,2024-03-02T18:05:00.000,2.0,3.0,0.0,False,4000938,ANGLE,CLEAR,DARK - LIGHTED,...,0 OPRY MILLS DR,NASHVILLE,TN,37214,9209,HERMIT,36.2037,-86.6956,TRUE,2024-03-02
3,20240148045,2024-03-02T17:55:00.000,2.0,0.0,0.0,False,4007311,FRONT TO REAR,CLEAR,DUSK,...,OLD HICKORY BLVD & DICKERSON PKE,NASHVILLE,TN,37207,2133,MADISO,36.2694,-86.7564,TRUE,2024-03-02
4,20240148007,2024-03-02T00:00:00.000,2.0,0.0,0.0,False,4010860,ANGLE,CLOUDY,DAYLIGHT,...,ELM HILL PIKE & EMERY DR,NASHVILLE,TN,37214,9105,HERMIT,36.1517,-86.677,TRUE,2024-03-02


Last, this analysis is only concerned with crashes in the vicinity of Main Street / Gallatin Avenue / Gallatin Pike in East Nashville & Madison.

In [10]:
crashes = crashes_raw[crashes_raw['street_address'].str.contains('GALLATIN|MAIN')]
crashes.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,hit_and_run,reporting_officer,collision_type_description,weather_description,illumination_description,...,street_address,city,state,zip,rpa,precinct,lat,long,property_damage,date
15,20240147685,2024-03-02T13:54:00.000,2.0,2.0,0.0,False,206964,ANGLE,CLOUDY,DAYLIGHT,...,HICKORY ST & GALLATIN PKES,MADISON,TN,37115,1605,MADISO,36.2604,-86.7129,TRUE,2024-03-02
18,20240147583,2024-03-02T11:20:00.000,2.0,0.0,0.0,True,4012266,ANGLE,CLOUDY,DAYLIGHT,...,GALLATIN PKES & MAPLE ST,MADISON,TN,37115,1607,MADISO,36.2617,-86.7124,TRUE,2024-03-02
42,20240146047,2024-03-01T20:07:00.000,2.0,1.0,0.0,False,4011186,FRONT TO REAR,RAIN,DARK - LIGHTED,...,MAIN ST & S 9TH ST,NASHVILLE,TN,37206,1999,EAST,36.176,-86.7572,TRUE,2024-03-01
45,20240145721,2024-03-01T18:07:00.000,3.0,0.0,0.0,True,4008019,FRONT TO REAR,RAIN,DARK - LIGHTED,...,GALLATIN PKEN & ALTUS AV,MADISON,TN,37115,1721,MADISO,36.2896,-86.7042,TRUE,2024-03-01
72,20240144690,2024-03-01T08:30:00.000,2.0,0.0,0.0,False,109536,ANGLE,RAIN,DAYLIGHT,...,GALLATIN AV & GRANADA AV,NASHVILLE,TN,37206,1319,EAST,36.1882,-86.7465,TRUE,2024-03-01


The resulting table is exported to a .csv file for use in the EDA notebook.

In [11]:
crashes.to_csv('../data/clean/crashes.csv')

Now repeat the process for active ROW permits.<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Active-Right-of-Way-Permits/a5tp-4w2v/about_data

In [12]:
row_permits_export = client.get("a5tp-4w2v", limit=2000000)
row_permits_raw = pd.DataFrame.from_records(row_permits_export)
row_permits_raw.head()

,initiated_date,permit,permit_type,permit_description,status,on_street,from_street,to_street,location_address,city,...,days_to_work,latitude,longitude,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,:@computed_region_b9k3_hpc2,company
0,2023-06-06T14:06:00.000,202320687,DRIVEWAY PERMIT,NEW DRIVEWAY,ACTIVE,849 BATTLE RD,MID-BLOCK,BATTLE RIDGE LN,849 BATTLE RD,NOLENSVILLE,...,Monday Tuesday Wednesday Thursday Friday Satur...,35.9746479630669,-86.6363976246735,"{'type': 'Point', 'coordinates': [-86.63639762...",1,2,29,18,3,NaN
1,2022-10-06T09:36:29.000,202236666,EXCAVATION PERMIT,STORMWATER INSTALL,ACTIVE,KINGS LN,PHIPPS DR,DRAKES BRANCH RD,KINGS LN,NASHVILLE,...,Tuesday Wednesday Thursday Friday Saturday,36.2175270092533,-86.846531521226,"{'type': 'Point', 'coordinates': [-86.84653152...",1,1,1,34,1,Middle Tennessee Infrastructure
2,2023-04-13T09:10:47.000,202313773,LANE CLOSURE,SEWER/WATER TAP,ACTIVE,4317 CATO RD,CATO RD,GILMORE CROSSING LN,4317 CATO RD,NASHVILLE,...,Monday Tuesday Wednesday Thursday Friday Satur...,36.2138889523239,-86.8666383650662,"{'type': 'Point', 'coordinates': [-86.86663836...",1,1,1,34,1,"R & A PLUMBING, LLC"
3,2023-05-15T10:55:27.000,202317854,EXCAVATION PERMIT,PLACING ANCHOR & POLE REMOVAL\nJOB# A02G7B0 (N...,ACTIVE,META CT,META DR,CUL-DE-SAC,META CT,NASHVILLE,...,Tuesday Wednesday Thursday Friday Saturday,36.0569811163057,-86.7352262648302,"{'type': 'Point', 'coordinates': [-86.73522626...",1,1,21,3,3,STAR CONSTRUCTION LLC
4,2023-07-10T14:37:22.000,202324681,DRIVEWAY PERMIT,MODIFACATION TO APRONS ENTRANCE,ACTIVE,417 417 WINDSOR DR,BLACKBURN AVE,HARDING PL,417 417 WINDSOR DR,NASHVILLE,...,Monday Tuesday Wednesday Thursday Friday Satur...,36.1014334222627,-86.8669816953158,"{'type': 'Point', 'coordinates': [-86.86698169...",1,1,32,5,6,THE OARENT COMPANY INC


In [13]:
row_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   initiated_date               22 non-null     object
 1   permit                       22 non-null     object
 2   permit_type                  22 non-null     object
 3   permit_description           22 non-null     object
 4   status                       22 non-null     object
 5   on_street                    22 non-null     object
 6   from_street                  22 non-null     object
 7   to_street                    22 non-null     object
 8   location_address             22 non-null     object
 9   city                         22 non-null     object
 10  zip                          22 non-null     object
 11  scope                        22 non-null     object
 12  scheduled_start              22 non-null     object
 13  scheduled_end                22 non-n

In [14]:
row_permits_raw = row_permits_raw.drop(row_permits_raw.columns[[4, 21, 22, 23, 24, 25]], axis=1)

In [15]:
row_permits_raw['initiated_date'] = pd.to_datetime(row_permits_raw['initiated_date'])
row_permits_raw['scheduled_start'] = pd.to_datetime(row_permits_raw['scheduled_start'])
row_permits_raw['scheduled_end'] = pd.to_datetime(row_permits_raw['scheduled_end'])

In [16]:
row_permits_raw['permit_type'] = row_permits_raw['permit_type'].astype(str).str.upper()
row_permits_raw['permit_description'] = row_permits_raw['permit_description'].astype(str).str.upper()
row_permits_raw['on_street'] = row_permits_raw['on_street'].astype(str).str.upper()
row_permits_raw['from_street'] = row_permits_raw['from_street'].astype(str).str.upper()
row_permits_raw['to_street'] = row_permits_raw['to_street'].astype(str).str.upper()
row_permits_raw['location_address'] = row_permits_raw['location_address'].astype(str).str.upper()
row_permits_raw['city'] = row_permits_raw['city'].astype(str).str.upper()
row_permits_raw['scope'] = row_permits_raw['scope'].astype(str).str.upper()
row_permits_raw['permit_applicant_name'] = row_permits_raw['permit_applicant_name'].astype(str).str.upper()
row_permits_raw['council_district'] = row_permits_raw['council_district'].astype(str).str.upper()
row_permits_raw['conditions_traffic_control'] = row_permits_raw['conditions_traffic_control'].astype(str).str.upper()
row_permits_raw['company'] = row_permits_raw['company'].astype(str).str.upper()

In [17]:
row_permits = row_permits_raw[row_permits_raw['on_street'].str.contains('GALLATIN|MAIN')]
row_permits

,initiated_date,permit,permit_type,permit_description,on_street,from_street,to_street,location_address,city,zip,...,scheduled_start,scheduled_end,permit_applicant_name,council_district,conditions_traffic_control,days_to_work,latitude,longitude,mapped_location,company


In [18]:
row_permits.to_csv('../data/clean/row_permits.csv')

...and Nashville 311 calls<br><br>
Source: https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh/about_data

In [19]:
nash_311_export = client.get("7qhx-rexh", limit=2000000)
nash_311_raw = pd.DataFrame.from_records(nash_311_export)
nash_311_raw.head()

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,...,:@computed_region_v67z_xm3t,:@computed_region_kh5x_g7w5,:@computed_region_cfa7_hbpz,:@computed_region_sjpq_96s8,:@computed_region_gisn_y5cm,:@computed_region_b9k3_hpc2,oem_id,contact_type,parent_case,preferred_language
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-05-04T17:54:30.000,2022-05-04T17:54:30.000,Phone,False,True,...,2,3,71,1527,4,2,NaN,NaN,NaN,NaN
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,2022-04-01T21:01:39.000,2022-04-05T11:29:54.000,Phone,False,False,...,1,9,1018,15550,4,1,NaN,NaN,NaN,NaN
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,2022-04-01T21:04:50.000,2022-04-06T20:14:15.000,Phone,False,False,...,8,33,258,24062,7,8,NaN,NaN,NaN,NaN
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-04-01T21:01:47.000,2022-04-01T21:01:47.000,Phone,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,2022-04-01T21:03:10.000,2022-04-01T22:04:31.000,Phone,False,False,...,7,13,50,30406,1,7,NaN,NaN,NaN,NaN


In [20]:
nash_311_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445370 entries, 0 to 1445369
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   case_number                  1445370 non-null  object
 1   status                       1445370 non-null  object
 2   case_request                 1445355 non-null  object
 3   case_subrequest              1444028 non-null  object
 4   additional_subrequest        1355568 non-null  object
 5   date_time_opened             1445370 non-null  object
 6   date_time_closed             1430565 non-null  object
 7   case_origin                  1445370 non-null  object
 8   state_issue                  1445370 non-null  object
 9   closed_when_created          1445370 non-null  object
 10  incident_address             983889 non-null   object
 11  incident_city                967168 non-null   object
 12  incident_council_district    968823 non-null   object
 1

In [21]:
nash_311_raw = nash_311_raw.drop(nash_311_raw.columns[[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]], axis=1)

In [22]:
nash_311_raw['date_time_opened'] = pd.to_datetime(nash_311_raw['date_time_opened'])
nash_311_raw['date_time_closed'] = pd.to_datetime(nash_311_raw['date_time_closed'])

In [23]:
nash_311_raw['status'] = nash_311_raw['status'].astype(str).str.upper()
nash_311_raw['case_request'] = nash_311_raw['case_request'].astype(str).str.upper()
nash_311_raw['case_subrequest'] = nash_311_raw['case_subrequest'].astype(str).str.upper()
nash_311_raw['additional_subrequest'] = nash_311_raw['additional_subrequest'].astype(str).str.upper()
nash_311_raw['case_origin'] = nash_311_raw['case_origin'].astype(str).str.upper()
nash_311_raw['state_issue'] = nash_311_raw['state_issue'].astype(str).str.upper()
nash_311_raw['incident_address'] = nash_311_raw['incident_address'].astype(str).str.upper()
nash_311_raw['incident_city'] = nash_311_raw['incident_city'].astype(str).str.upper()
nash_311_raw['incident_council_district'] = nash_311_raw['incident_council_district'].astype(str).str.upper()
nash_311_raw['contact_type'] = nash_311_raw['contact_type'].astype(str).str.upper()
nash_311_raw['parent_case'] = nash_311_raw['parent_case'].astype(str).str.upper()
nash_311_raw['preferred_language'] = nash_311_raw['preferred_language'].astype(str).str.upper()

In [24]:
nash_311 = nash_311_raw[nash_311_raw['incident_address'].str.contains('GALLATIN|MAIN')]
nash_311

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,contact_type,parent_case,preferred_language
145,706362,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2021-06-14 03:41:00,2021-06-14 03:41:00,MOBILE APP,FALSE,TRUE,609 GALLATIN PIKE S,MADISON,9,37115,36.2568189,-86.7137494,"{'latitude': '36.2568189', 'longitude': '-86.7...",NAN,NAN,NAN
195,705132,CLOSED,"TRASH, RECYCLING & LITTER",MISSED PICKUP - RECYCLE CART SERVICE,RECYCLE - BACKDOOR,2021-06-11 02:40:00,2021-06-23 15:29:00,HUBNASHVILLE COMMUNITY,FALSE,FALSE,630 MAINSTREAM DR,NASHVILLE,2,37228,36.19805001,-86.81561503,"{'latitude': '36.19805001', 'longitude': '-86....",NAN,NAN,NAN
238,932992,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2022-04-01 22:07:35,2022-04-01 22:07:35,MOBILE APP,FALSE,True,2232 GALLATIN PIKE N,MADISON,10,37115,36.3078416,-86.6849309,"{'latitude': '36.3078416', 'longitude': '-86.6...",NAN,NAN,NAN
263,707700,CLOSED,"STREETS, ROADS & SIDEWALKS",TRAFFIC LIGHT TIMING,TRAFFIC LIGHT TIMING,2021-06-15 14:40:00,2021-06-28 20:16:00,PHONE,FALSE,FALSE,CONFERENCE DR & GALLATIN PIKE N,MADISON,10,37115,36.3027491,-86.6918546,"{'latitude': '36.3027491', 'longitude': '-86.6...",NAN,NAN,NAN
426,932994,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2022-04-01 22:11:51,2022-04-01 22:11:51,MOBILE APP,FALSE,True,2232 GALLATIN PIKE N,MADISON,10,37115,36.3078416,-86.6849309,"{'latitude': '36.3078416', 'longitude': '-86.6...",NAN,NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445207,1499417,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2024-03-04 02:25:56,2024-03-04 02:25:56,HUBNASHVILLE COMMUNITY,FALSE,True,1764 GALLATIN PIKE N,MADISON,10,37115,36.2969032116639,-86.6992787,"{'latitude': '36.2969032116639', 'longitude': ...",NAN,NAN,NAN
1445250,604899,IN PROGRESS,"STREETS, ROADS & SIDEWALKS",REQUEST NEW SIGN,REQUEST NEW SIGN,2021-03-01 15:11:05,NaT,PHONE,FALSE,False,GALLATIN PIKE & W EASTLAND AVE,NAN,NAN,NaN,36.3911548,-86.4478856,"{'latitude': '36.3911548', 'longitude': '-86.4...",NAN,NAN,NAN
1445257,791695,IN PROGRESS,"STREETS, ROADS & SIDEWALKS",ROADWORK COMPLAINT,ROADWORK COMPLAINT,2021-09-29 17:43:09,NaT,HUBNASHVILLE COMMUNITY,FALSE,False,MAIN ST,NASHVILLE,19,37219,36.163,-86.782,"{'latitude': '36.163', 'longitude': '-86.782'}",NAN,NAN,NAN
1445288,918328,IN PROGRESS,"STREETS, ROADS & SIDEWALKS",TRAFFIC ENGINEERING,REQUEST MARKINGS,2022-03-10 13:37:05,NaT,MOBILE APP,FALSE,False,516 MAIN ST,NASHVILLE,6,37206,36.174824076343114,-86.76011480391026,"{'latitude': '36.174824076343114', 'longitude'...",NAN,NAN,NAN


In [25]:
nash_311.to_csv('../data/clean/nash_311.csv')

...and building permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7/about_data

In [26]:
bdlg_permits_export = client.get("3h5w-q8b7", limit=2000000)
bdlg_permits_raw = pd.DataFrame.from_records(bdlg_permits_export)
bdlg_permits_raw.head()

,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,state,...,:@computed_region_v3ji_vzam,:@computed_region_c9xn_skx3,:@computed_region_sjpq_96s8,:@computed_region_kh5x_g7w5,:@computed_region_yf9r_ed6g,:@computed_region_fvtq_wnma,:@computed_region_p6sk_2acq,:@computed_region_b9k3_hpc2,:@computed_region_gxvr_9jxz,:@computed_region_cfa7_hbpz
0,2020003022,Building Commercial - New,"Multifamily, Tri-Plex, Quad, Apartments",05100001800,2020-01-15T00:00:00.000,2021-11-19T00:00:00.000,440536,808 SKYLINE RIDGE DR 200,MADISON,TN,...,2005,3,1527,3,3,26,3,2,12,NaN
1,2020008381,Building Use & Occupancy,"Multifamily, Townhome",05100018100,2020-02-06T00:00:00.000,2021-11-24T00:00:00.000,1,609 CREATIVE WAY,MADISON,TN,...,2005,3,1527,3,3,26,3,2,12,NaN
2,2020010440,Building Commercial - Rehab,"Retail, Department / Retail Stores",10700005000,2020-02-18T00:00:00.000,2021-03-16T00:00:00.000,242277,1 TERMINAL DR,NASHVILLE,TN,...,2005,18,15807,18,18,44,6,9,38,NaN
3,2020014026,Building Commercial - Rehab,"Hospice, Institutional I-2",09211039000,2020-03-04T00:00:00.000,2021-04-01T00:00:00.000,420926,2300 PATTERSON ST,NASHVILLE,TN,...,2005,15,30406,15,15,25,25,7,40,NaN
4,2020024429,Building Use & Occupancy,"Multifamily, Apt / Twnhome > 5 Unit Bldg",103080O90000CO,2020-04-17T00:00:00.000,2021-07-20T00:00:00.000,1,4601 C SLOAN RD,NASHVILLE,TN,...,2005,22,15798,22,22,12,10,5,46,24


In [27]:
bdlg_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33592 entries, 0 to 33591
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       33592 non-null  object
 1   permit_type_description      33592 non-null  object
 2   permit_subtype_description   33592 non-null  object
 3   parcel                       33592 non-null  object
 4   date_entered                 33592 non-null  object
 5   date_issued                  33592 non-null  object
 6   const_cost                   33591 non-null  object
 7   address                      33592 non-null  object
 8   city                         33591 non-null  object
 9   state                        33592 non-null  object
 10  zip                          33591 non-null  object
 11  subdivision_lot              33591 non-null  object
 12  contact                      33592 non-null  object
 13  per_ty                       33

In [28]:
bdlg_permits_raw = bdlg_permits_raw.drop(bdlg_permits_raw.columns[[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]], axis=1)

In [29]:
bdlg_permits_raw['date_entered'] = pd.to_datetime(bdlg_permits_raw['date_entered'])
bdlg_permits_raw['date_issued'] = pd.to_datetime(bdlg_permits_raw['date_issued'])

In [30]:
bdlg_permits_raw['permit_type_description'] = bdlg_permits_raw['permit_type_description'].astype(str).str.upper()
bdlg_permits_raw['permit_subtype_description'] = bdlg_permits_raw['permit_subtype_description'].astype(str).str.upper()
bdlg_permits_raw['address'] = bdlg_permits_raw['address'].astype(str).str.upper()
bdlg_permits_raw['city'] = bdlg_permits_raw['city'].astype(str).str.upper()
bdlg_permits_raw['state'] = bdlg_permits_raw['state'].astype(str).str.upper()
bdlg_permits_raw['contact'] = bdlg_permits_raw['contact'].astype(str).str.upper()
bdlg_permits_raw['per_ty'] = bdlg_permits_raw['per_ty'].astype(str).str.upper()
bdlg_permits_raw['permit_subtype'] = bdlg_permits_raw['permit_subtype'].astype(str).str.upper()
bdlg_permits_raw['purpose'] = bdlg_permits_raw['purpose'].astype(str).str.upper()
bdlg_permits_raw['council_dist'] = bdlg_permits_raw['council_dist'].astype(str).str.upper()
bdlg_permits_raw['census_tract'] = bdlg_permits_raw['census_tract'].astype(str).str.upper()
bdlg_permits_raw['case_status'] = bdlg_permits_raw['case_status'].astype(str).str.upper()

In [31]:
bdlg_permits = bdlg_permits_raw[bdlg_permits_raw['address'].str.contains('GALLATIN|MAIN')].reset_index()
bdlg_permits

,index,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,...,subdivision_lot,contact,per_ty,permit_subtype,ivr_trk,purpose,council_dist,census_tract,case_status,mapped_location
0,32,2020079033,BUILDING COMMERCIAL - REHAB,"PERSONAL CARE SVCS, DRY CLEAN & LAUNDRY",03406000200,2020-12-18,2021-03-02,200000,1699 GALLATIN PIKE N,MADISON,...,LOT 2 RIVERGATE STATION SEC 1 2ND RESUB LOT 2,"AVENUE CONSTRUCTION, LLC",CACR,CAF18B008,3907564,DEMOLITION OF FORMER RESTAURANT SPACE TO CONVE...,10,37010401,DONE,"{'latitude': '36.293865007', 'longitude': '-86..."
1,51,2021005245,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,08309013400,2021-01-26,2021-03-18,4250,1000 MAIN ST,NASHVILLE,...,LOTS 14 & 16 P/O LOT 15 BRANSFORD AND MCWHIRTE...,AAA SIGN ME UP LLC,CASN,CAZ02A001,3919103,TO ERECT 15SF ILLUMINATED PROJECTING SIGN AND ...,6,37019200,EXPIRED,"{'latitude': '36.177996009', 'longitude': '-86..."
2,63,2021008120,BUILDING COMMERCIAL - REHAB,"MANUFACTURING, LIGHT HAZARDOUS (H1-H5)",07210009700,2021-02-08,2021-03-25,150000,2703 GALLATIN PIKE,NASHVILLE,...,PT LOT 3 EASTHILL SUB,"CERTIFIED CONSTRUCTION SERVICES, LLC",CACR,CAH06H000,3923401,TENANT IMPROVEMENT OF BUILDNIG AT 2703 GALLATA...,5,37011400,DONE,"{'latitude': '36.199367012', 'longitude': '-86..."
3,65,2021010474,BUILDING COMMERCIAL - ADDITION,ADDITION,05100015900,2021-02-22,2021-08-24,138898,5110 GALLATIN PIKE,MADISON,...,E SIDE GALLATIN PIKE S OF BERKLEY DRIVE,MCCLESKEY CONSTRUCTION COMPANY,CACA,ADDITION,3926808,TO CONSTRUCT A 1490 SF ADDITION TO EXISTING MA...,7,37010702,DONE,"{'latitude': '36.236840683', 'longitude': '-86..."
4,73,2021011848,BUILDING COMMERCIAL - NEW,"MULTIFAMILY, APT / TWNHOME > 5 UNIT BLDG",08212031300,2021-02-27,2021-09-09,2025671,918 MAIN ST,NASHVILLE,...,PT LOTS 5 & 6 R W BROWN ADDN,"POWELL ARCHITECTURE & BUILDING STUDIO, LLC",CACN,CAA03R399,3933909,"MUG-A, UDO, MDHA, UZO TO CONSTRUCT MIXED USE ...",6,37019200,DONE,"{'latitude': '36.176308019', 'longitude': '-86..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,33256,2023070739,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,05100018000,2023-09-21,2023-09-27,4000,1504 GALLATIN PIKE S,MADISON,...,S. SIDE WALTON LANE W. OF GALLATIN PIKE,JARVIS AWARD SIGN AND FLAG CO,CASN,CAZ02A001,4386367,"TO ERECT 1 ILLUMINATED WALL SIGN, TO ADD 2 TEN...",7,37010702,ISSUED,"{'latitude': '36.239915874', 'longitude': '-86..."
457,33475,2023087993,BUILDING COMMERCIAL - REHAB,"PERSONAL CARE SVCS, HAIR/ NAIL SALON",04212007300,2023-11-09,2024-01-05,20000,115 GALLATIN PIKE N,MADISON,...,N SIDE OLD HICKORY BLVD W OF GALLATIN PK,DEBRIA WALLER,CACR,CAF18B012,4412171,TO CONDUCT RENOVATIONS TO EXISTING BUILDING FO...,9,37010402,ISSUED,"{'latitude': '36.264543015', 'longitude': '-86..."
458,33506,2023078977,BUILDING COMMERCIAL - REHAB,RESTAURANT (FAST FOOD),05100018000,2023-10-16,2024-01-31,300000,1504 B GALLATIN PIKE S,MADISON,...,S. SIDE WALTON LANE W. OF GALLATIN PIKE,HALIE CHANDLER,CACR,CAF30A203,4398640,TO CONDUCT RENOVATIONS TO 1811SF OF EXISTING B...,7,37010702,ISSUED,"{'latitude': '36.239915874', 'longitude': '-86..."
459,33535,2023102128,BUILDING COMMERCIAL - REHAB,"RETAIL, DRUG STORES",08305008700,2023-12-27,2024-02-13,2000,710 GALLATIN AVE,NASHVILLE,...,LOTS 9 10 BAXTER SUB EASTWOOD,SELF CONTRACTOR COMMERCIAL (SEE APPLICANT INFO...,CACR,CAF20M002,4433297,"USE AND OCCUPANCY ONLY, NO CONSTRUCTION PER AP...",5,37011700,ISSUED,"{'latitude': '36.184247986', 'longitude': '-86..."


Need the coordinate data from the `mapped location` column

In [32]:
bdlg_permits['lat'] = [bdlg_permits['mapped_location'][x].get('latitude') for x in range(0, len(bdlg_permits))]
bdlg_permits['long'] = [bdlg_permits['mapped_location'][x].get('longitude') for x in range(0, len(bdlg_permits))]

In [33]:
bdlg_permits.to_csv('../data/clean/bldg_permits.csv')

...and Short Term Rental Permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Residential-Short-Term-Rental-Permits/2z82-v8pm/about_data

In [34]:
str_export = client.get("2z82-v8pm", limit=100000)
strs_raw = pd.DataFrame.from_records(str_export)
strs_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15188 entries, 0 to 15187
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       15188 non-null  object
 1   applicant                    15173 non-null  object
 2   contact                      14792 non-null  object
 3   permit_subtype_description   15188 non-null  object
 4   permit_status                15188 non-null  object
 5   parcel                       15188 non-null  object
 6   date_entered                 15188 non-null  object
 7   date_issued                  13903 non-null  object
 8   expiration_date              15184 non-null  object
 9   address                      15188 non-null  object
 10  city                         15187 non-null  object
 11  state                        15187 non-null  object
 12  zip                          15188 non-null  object
 13  subdivision_lot              15

In [35]:
strs_raw = strs_raw.drop(strs_raw.columns[[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]], axis=1)

In [36]:
strs_raw['date_entered'] = pd.to_datetime(strs_raw['date_entered'])
strs_raw['date_issued'] = pd.to_datetime(strs_raw['date_issued'])
strs_raw['expiration_date'] = pd.to_datetime(strs_raw['expiration_date'])

In [37]:
strs_raw['applicant'] = strs_raw['applicant'].astype(str).str.upper()
strs_raw['contact'] = strs_raw['contact'].astype(str).str.upper()
strs_raw['permit_subtype_description'] = strs_raw['permit_subtype_description'].astype(str).str.upper()
strs_raw['permit_status'] = strs_raw['permit_status'].astype(str).str.upper()
strs_raw['address'] = strs_raw['address'].astype(str).str.upper()
strs_raw['city'] = strs_raw['city'].astype(str).str.upper()
strs_raw['state'] = strs_raw['state'].astype(str).str.upper()
strs_raw['permit_subtype'] = strs_raw['permit_subtype'].astype(str).str.upper()
strs_raw['purpose'] = strs_raw['purpose'].astype(str).str.upper()
strs_raw['permit_owner_name'] = strs_raw['permit_owner_name'].astype(str).str.upper()
strs_raw['permit_owner_addr2'] = strs_raw['permit_owner_addr2'].astype(str).str.upper()
strs_raw['permit_owner_city'] = strs_raw['permit_owner_city'].astype(str).str.upper()
strs_raw['permit_owner_state'] = strs_raw['permit_owner_state'].astype(str).str.upper()
strs_raw['permit_type'] = strs_raw['permit_type'].astype(str).str.upper()
strs_raw['permit_subtype'] = strs_raw['permit_subtype'].astype(str).str.upper()
strs_raw['council_dist'] = strs_raw['council_dist'].astype(str).str.upper()
strs_raw['census_tract'] = strs_raw['census_tract'].astype(str).str.upper()
strs_raw['permit_owner_addr1'] = strs_raw['permit_owner_addr1'].astype(str).str.upper()
strs_raw['permit_owner_addr3'] = strs_raw['permit_owner_addr3'].astype(str).str.upper()

In [38]:
rentals = strs_raw[strs_raw['zip'].isin(['37206', '37216', '37115'])].reset_index()
rentals

,index,permit,applicant,contact,permit_subtype_description,permit_status,parcel,date_entered,date_issued,expiration_date,...,permit_owner_city,permit_owner_state,permit_owner_zip,permit_type,permit_subtype,council_dist,census_tract,mapped_location,permit_owner_addr1,permit_owner_addr3
0,1,T2017008702,"TREHY, KEITH","TREHY, KEITH",SHORT TERM RENTAL - NON-OWNER OCCUPIED,EXPIRED,08304007300,2017-02-06,NaT,2017-05-08,...,NASHVILLE,TN,37206,CASR,CAZ10A002,7,37011600,"{'latitude': '36.189219', 'longitude': '-86.71...",NAN,NAN
1,2,2017055846,"HALE, JEFFREY L.",JEFFREY HALE,SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,08212026000,2017-08-25,2017-09-01,2024-09-01,...,HENDERSONVILLE,TN,37075,CASR,CAZ10A001,5,37011900,"{'latitude': '36.178000996', 'longitude': '-86...",NAN,NAN
2,16,2023047359,"OWNBEY, JASON","OWNBEY, JASON",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,09402023100,2023-07-19,2023-07-19,2024-07-19,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012200,"{'latitude': '36.166715015', 'longitude': '-86...",NAN,NAN
3,17,2022030944,"BURKART, JAMES","BURKART, JAMES",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,06114015300,2022-05-10,2022-05-10,2024-05-10,...,NASHVILLE,TN,37216,CASR,CAZ10A001,7,37011200,"{'latitude': '36.214595002', 'longitude': '-86...",NAN,NAN
4,21,2019021499,SARA CLINE,SARA CLINE,SHORT TERM RENTAL - OWNER OCCUPIED,EXPIRED,08208037200,2019-04-12,2019-04-19,2020-04-19,...,NASHVILLE,TN,37206,CASR,CAZ10A001,5,37011700,"{'latitude': '36.185369988', 'longitude': '-86...",NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2867,15133,2021051008,"11TH AVENUE PARTNERS, LLC",DWELL NASHVILLE,SHORT TERM RENTAL � MULTIFAMILY,ISSUED,082121D00200CO,2021-08-10,2021-08-13,2024-08-13,...,NASHVILLE,TN,37220,CASR,CAZ10A003,5,37011900,"{'latitude': '36.179004016', 'longitude': '-86...",BRAD THOMASON,NAN
2868,15135,2020002749,"STRATON, ANDREW & MARIKO","STRATON, ANDREW & MARIKO",SHORT TERM RENTAL - OWNER OCCUPIED,CANCELLED,08305036900,2020-01-14,2020-02-26,2023-02-26,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012100,"{'latitude': '36.184620017', 'longitude': '-86...",NAN,NAN
2869,15138,T2017030096,"NELSON, CANDICE & MATTHEW",CANDICE NELSON,SHORT TERM RENTAL - OWNER OCCUPIED,CANCELLED,08314008300,2017-05-05,NaT,2017-08-03,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012200,"{'latitude': '36.174626987', 'longitude': '-86...",NAN,NAN
2870,15149,2023037519,"MOORE, JONATHAN E. & DONNA KIMBERLY & ET AL","MOORE, JONATHAN E. & DONNA KIMBERLY & ET AL",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,05207008600,2023-06-07,2023-06-07,2024-06-07,...,MADISON,TN,37115,CASR,CAZ10A001,9,37010601,"{'latitude': '36.247076009', 'longitude': '-86...",NAN,NAN


Need the coordinate data from the `mapped location` column

In [39]:
rentals['lat'] = [rentals['mapped_location'][x].get('latitude') for x in range(0, len(rentals))]
rentals['long'] = [rentals['mapped_location'][x].get('longitude') for x in range(0, len(rentals))]

In [40]:
rentals.to_csv('../data/clean/rentals.csv')

...and the pedestrian signal inventory<br><br>
Source: https://data.nashville.gov/Transportation/Pedestrian-Signal-Inventory-and-ADA-Self-Assessmen/6xet-f7u7/about_data

In [41]:
ped_sig = client.get("6xet-f7u7", limit=1000000)
ped_inv = pd.DataFrame.from_records(ped_sig)
ped_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3153 entries, 0 to 3152
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   intersectionid               3153 non-null   object
 1   onst                         3153 non-null   object
 2   crst                         3153 non-null   object
 3   quad                         3151 non-null   object
 4   inspector                    3153 non-null   object
 5   date                         3153 non-null   object
 6   evnt_lat                     3136 non-null   object
 7   evnt_lon                     3136 non-null   object
 8   evnt_map                     3153 non-null   object
 9   evnt_type                    3150 non-null   object
 10  ped_signal_pres              3150 non-null   object
 11  oa_cmnt                      3150 non-null   object
 12  mapped_location              3136 non-null   object
 13  :@computed_region_wvby_4s8j  3136

In [42]:
ped_inv = ped_inv.drop(ped_inv.columns[[13, 14, 15, 16]], axis=1)

In [43]:
ped_inv['onst'] = ped_inv['onst'].astype(str).str.upper()
ped_inv['crst'] = ped_inv['crst'].astype(str).str.upper()
ped_inv['quad'] = ped_inv['quad'].astype(str).str.upper()
ped_inv['inspector'] = ped_inv['inspector'].astype(str).str.upper()
ped_inv['evnt_type'] = ped_inv['evnt_type'].astype(str).str.upper()

In [44]:
ped_inv = ped_inv[(ped_inv['onst'].str.contains('GALLATIN|MAIN')) | (ped_inv['crst'].str.contains('GALLATIN|MAIN'))]

In [45]:
ped_inv['date'] = pd.to_datetime(ped_inv['date'])

In [46]:
ped_inv.to_csv('../data/clean/ped_inv.csv')